# Setup (RUN EVERYTIME)

In [ ]:
# Use if Drive mounting bugs out
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

# NOTE 1: The '%' symbolizes special "magic" commands for Jupyter/IPython
# NOTE 2: This specific ("magic") command changes the current directory...
          # within the notebook kernel
%cd /content/drive/MyDrive/DATASCI266
# print(os.getcwd()) # No need to print

/content/drive/MyDrive/DATASCI266


# Get NewsQA dataset (DONE)

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# MRQA-style NewsQA; works with current datasets lib
newsqa = load_dataset("lucadiliello/newsqa")

train_ds = newsqa["train"]
val_ds   = newsqa["validation"]

# If the context column is called "paragraph", rename it to "context"
if "paragraph" in train_ds.column_names and "context" not in train_ds.column_names:
    train_ds = train_ds.rename_column("paragraph", "context")
    val_ds   = val_ds.rename_column("paragraph", "context")

# Convert to RACE-style Multiple Choice (MC)

In [ ]:
import random
from collections import defaultdict

def convert_newsqa_to_race_style(newsqa_dataset):
    """
    Convert lucadiliello/newsqa into RACE-style MC examples.
    Answers are lists of strings (unlike SQuAD dicts).
    """
    processed_examples = []

    # STEP 1: group QAs by context (like paragraphs)
    context_to_qas = defaultdict(list)
    for ex in newsqa_dataset:
        context = ex["context"]
        context_to_qas[context].append(ex)

    # STEP 2: iterate over each context group
    for context, qas in context_to_qas.items():

        # gather all unique answers from this context (wrong answer pool)
        new_options = []
        for qa in qas:
            answers = qa.get("answers", [])
            for ans in answers:
                ans = ans.strip()
                if ans and ans not in new_options:
                    new_options.append(ans)

        # convert each QA
        for qa in qas:
            answers = qa.get("answers", [])
            if not answers:
                continue

            question = qa["question"]
            correct_answer = answers[0].strip()
            if not correct_answer:
                continue

            # collect wrong answers
            wrong_answers = [
                ans for ans in new_options if ans != correct_answer
            ][:3]

            if len(wrong_answers) < 3:
                continue

            options = [correct_answer] + wrong_answers
            random.shuffle(options)
            label = options.index(correct_answer)

            mc_example = {
                "context": context,
                "query": question,
                "option_0": options[0],
                "option_1": options[1],
                "option_2": options[2],
                "option_3": options[3],
                "label": label
            }

            processed_examples.append(mc_example)

    return processed_examples

In [ ]:
newsqa_mc_train = convert_newsqa_to_race_style(train_ds)
newsqa_mc_val = convert_newsqa_to_race_style(val_ds)

print("Train MC examples:", len(newsqa_mc_train))
print("Val MC examples:", len(newsqa_mc_val))
print(newsqa_mc_train[0])


Train MC examples: 69501
Val MC examples: 3929
{'context': 'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors" by the In

In [ ]:
from sklearn.model_selection import train_test_split

# Split NewsQA MC validation into val + test
newsqa_val, newsqa_test = train_test_split(
    newsqa_mc_val,
    test_size=0.2,
    random_state=42,
)

print("New val size:", len(newsqa_val))
print("New test size:", len(newsqa_test))

New val size: 3143
New test size: 786


In [ ]:
import os
import json

save_path = "/content/drive/MyDrive/DATASCI266/NewsQA"
os.makedirs(save_path, exist_ok=True)

def save_mc(name, data):
    with open(f"{save_path}/{name}.jsonl", "w", encoding="utf-8") as f:
        for ex in data:
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")
    print(f"Saved {len(data)} → {save_path}/{name}.jsonl")

save_mc("train", newsqa_mc_train)    # already generated
save_mc("val", newsqa_val)           # new validation split
save_mc("test", newsqa_test)         # new test split


Saved 69501 → /content/drive/MyDrive/DATASCI266/NewsQA/train.jsonl
Saved 3143 → /content/drive/MyDrive/DATASCI266/NewsQA/val.jsonl
Saved 786 → /content/drive/MyDrive/DATASCI266/NewsQA/test.jsonl


# Finetuning (DONE)

In [ ]:
# Change to desired working directory
%cd /content/drive/MyDrive/DATASCI266/lrqa

/content/drive/MyDrive/DATASCI266/lrqa


In [ ]:
!export WANDB_DISABLED=true TF_CPP_MIN_LOG_LEVEL=2 PYTHONUNBUFFERED=1; \
python -u -m lrqa.run_lrqa \
  --model_name_or_path "/content/drive/MyDrive/DATASCI266/models/roberta_base_0" \
  --model_mode mc \
  --max_seq_length 512 \
  --task_name custom \
  --task_base_path "/content/drive/MyDrive/DATASCI266/NewsQA" \
  --output_dir "/content/drive/MyDrive/DATASCI266/NewsQA/models/roberta_base_3_finetuning/" \
  --learning_rate 1e-5 \
  --num_train_epochs 3 \
  --warmup_ratio 0.1 \
  --eval_steps 1000 \
  --save_steps 1000 \
  --save_total_limit 5 \
  --save_strategy steps \
  --eval_strategy steps \
  --load_best_model_at_end \
  --per_device_train_batch_size 8 \
  --per_device_eval_batch_size 8 \
  --gradient_accumulation_steps 4 \
  --do_train --do_eval --do_predict --predict_phases validation \
  --logging_strategy steps --logging_steps 50

2025-11-28 04:38:24.267495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764304704.287216    1152 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764304704.293001    1152 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764304704.309065    1152 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764304704.309093    1152 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764304704.309099    1152 computation_placer.cc:177] computation placer alr

# Evaluation with BBQ Dataset (DONE)

In [ ]:
# Change to desired working directory
%cd /content/drive/MyDrive/DATASCI266/lrqa

/content/drive/MyDrive/DATASCI266


In [ ]:
%%bash

cd /content/drive/MyDrive/DATASCI266/lrqa

MODEL_LOCATION=/content/drive/MyDrive/DATASCI266/NewsQA/models/roberta_base_3_finetuning/checkpoint-last
BBQ_LOCATION=/content/drive/MyDrive/DATASCI266/RACE_preproc/bbq
RUN_RESULTS_LOCATION=/content/drive/MyDrive/DATASCI266/NewsQA/bbq_runs
BATCH_SIZE=8

# To avoid warnings
export TF_CPP_MIN_LOG_LEVEL=2
export WANDB_DISABLED=true

for CATEGORY in Age Disability_status Gender_identity Nationality Physical_appearance Race_ethnicity Race_x_SES Race_x_gender Religion SES Sexual_orientation; do
  echo "Running BBQ category: ${CATEGORY}"
  mkdir -p "${RUN_RESULTS_LOCATION}/${CATEGORY}"

  python -m lrqa.run_lrqa \
    --model_name_or_path "${MODEL_LOCATION}" \
    --model_mode mc \
    --max_seq_length 512 \
    --task_name custom \
    --task_base_path "${BBQ_LOCATION}/${CATEGORY}" \
    --output_dir "${RUN_RESULTS_LOCATION}/${CATEGORY}" \
    --per_device_eval_batch_size ${BATCH_SIZE} \
    --do_eval --do_predict --predict_phases validation \
    --report_to none
done

Running BBQ category: Age
🚀 Starting fresh (no checkpoint detected).
{
  "eval_loss": 3.5344622135162354,
  "eval_model_preparation_time": 0.0024,
  "eval_accuracy": 0.280434787273407,
  "eval_runtime": 279.7672,
  "eval_samples_per_second": 13.154,
  "eval_steps_per_second": 1.644
}
Running BBQ category: Disability_status
🚀 Starting fresh (no checkpoint detected).
{
  "eval_loss": 3.2480738162994385,
  "eval_model_preparation_time": 0.0024,
  "eval_accuracy": 0.2744216024875641,
  "eval_runtime": 120.2683,
  "eval_samples_per_second": 12.938,
  "eval_steps_per_second": 1.621
}
Running BBQ category: Gender_identity
🚀 Starting fresh (no checkpoint detected).
{
  "eval_loss": 3.7092859745025635,
  "eval_model_preparation_time": 0.0054,
  "eval_accuracy": 0.31805360317230225,
  "eval_runtime": 436.1385,
  "eval_samples_per_second": 13.005,
  "eval_steps_per_second": 1.626
}
Running BBQ category: Nationality
🚀 Starting fresh (no checkpoint detected).
{
  "eval_loss": 3.0633034706115723,
  

2025-12-03 08:45:17.674772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764751517.694511    1102 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764751517.700419    1102 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764751517.715293    1102 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764751517.715315    1102 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764751517.715318    1102 computation_placer.cc:177] computation placer alr